# Phase 1: Data Preprocessing & Feature Engineering

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer

In [2]:
# Load the dataset
df = pd.read_csv('marketing_campaign.csv', sep='\t')  # Note: Adjust delimiter if needed
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


## Data Cleaning

In [3]:
# Handle missing values (e.g., Income)
df['Income'] = df['Income'].fillna(df['Income'].median())

# Remove outliers (e.g., Income > 99th percentile)
income_threshold = df['Income'].quantile(0.99)
df = df[df['Income'] <= income_threshold]

# Drop redundant columns (Z_CostContact, Z_Revenue are constants)
df = df.drop(columns=['Z_CostContact', 'Z_Revenue'])

In [4]:
df.shape

(2217, 27)

## Feature Encoding

In [5]:
# Ordinal encoding for Education (assuming ordinality)
education_order = ['Basic', '2n Cycle', 'Graduation', 'Master', 'PhD']
df['Education'] = df['Education'].astype('category').cat.set_categories(education_order, ordered=True)
df['Education_encoded'] = df['Education'].cat.codes

# One-hot encoding for Marital_Status
df = pd.get_dummies(df, columns=['Marital_Status'], prefix='Marital')

In [6]:
df.shape

(2217, 35)

In [7]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Income', 'Kidhome', 'Teenhome',
       'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain', 'Response', 'Education_encoded', 'Marital_Absurd',
       'Marital_Alone', 'Marital_Divorced', 'Marital_Married',
       'Marital_Single', 'Marital_Together', 'Marital_Widow', 'Marital_YOLO'],
      dtype='object')

In [8]:
df['Education']

0       Graduation
1       Graduation
2       Graduation
3       Graduation
4              PhD
           ...    
2235    Graduation
2236           PhD
2237    Graduation
2238        Master
2239           PhD
Name: Education, Length: 2217, dtype: category
Categories (5, object): ['Basic' < '2n Cycle' < 'Graduation' < 'Master' < 'PhD']

In [9]:
df['Education_encoded']

0       2
1       2
2       2
3       2
4       4
       ..
2235    2
2236    4
2237    2
2238    3
2239    4
Name: Education_encoded, Length: 2217, dtype: int8

## New Feature Creation

In [10]:
# Convert Dt_Customer to tenure (days since enrollment)
# We calculate from the last date of purchase, not today (2025)
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
last_purchased_date = df['Dt_Customer'].max()
print('last_purchased_date',last_purchased_date)
df['Customer_Tenure'] = (last_purchased_date - df['Dt_Customer']).dt.days

# Derive "Family_Size" (Kidhome + Teenhome)
df['Family_Size'] = df['Kidhome'] + df['Teenhome']

# Derive "Total_Spend" (sum of all product purchases)
product_columns = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_Spend'] = df[product_columns].sum(axis=1)

last_purchased_date 2014-06-29 00:00:00


In [11]:
df['Customer_Tenure'].head()

0    663
1    113
2    312
3    139
4    161
Name: Customer_Tenure, dtype: int64

In [12]:
df['Family_Size'].head()

0    0
1    2
2    0
3    1
4    1
Name: Family_Size, dtype: int64

In [13]:
df['Total_Spend'].head()

0    1617
1      27
2     776
3      53
4     422
Name: Total_Spend, dtype: int64

## Scaling & Correlation

In [14]:
# Scale numerical features (for clustering/autoencoder)
scaler = StandardScaler()
numerical_features = ['Income', 'Customer_Tenure', 'Total_Spend', 'Family_Size', 'Recency']
df[numerical_features] = scaler.fit_transform(df[numerical_features])
# Correlation analysis (optional, for EDA)
correlation_matrix = df[numerical_features].corr()

In [15]:
# Note: save scaler for the phase 3 reuse
import joblib

# Save the scaler to a file
joblib.dump(scaler, 'scaler.joblib')  # Saves to disk

['scaler.joblib']

In [16]:
df[numerical_features] 

,Income,Customer_Tenure,Total_Spend,Family_Size,Recency
0,0.336654,1.530151,1.704107,-1.275199,0.305137
1,-0.244749,-1.191341,-0.958740,1.390634,-0.386181
2,1.000925,-0.206656,0.295645,-1.275199,-0.800972
3,-1.215793,-1.062688,-0.915196,0.057718,-0.800972
4,0.344295,-0.953829,-0.297215,0.057718,1.549510
...,...,...,...,...,...
2235,0.488734,0.134768,1.241877,0.057718,-0.109654
2236,0.626320,-1.656468,-0.260371,2.723550,0.236005
2237,0.279618,-0.983518,1.074402,-1.275199,1.445812
2238,0.884191,-0.978570,0.407853,0.057718,-1.423158


## Documentation & Output 

In [17]:
# Save preprocessed data
df.to_csv('preprocessed_campaign_data.csv', index=False)

# Print summary of changes
print("Preprocessing Summary:")
print(f"- Missing values handled: Income imputed with median.")
print(f"- New features added: Customer_Tenure, Family_Size, Total_Spend.")
print(f"- Categorical features encoded: Education (ordinal), Marital_Status (one-hot).")
print(f"- Numerical features scaled: {numerical_features}")

Preprocessing Summary:
- Missing values handled: Income imputed with median.
- New features added: Customer_Tenure, Family_Size, Total_Spend.
- Categorical features encoded: Education (ordinal), Marital_Status (one-hot).
- Numerical features scaled: ['Income', 'Customer_Tenure', 'Total_Spend', 'Family_Size', 'Recency']
